Now that your CycleGAN model has been trained, you can use it to generate Monet-style images from photos in the specified directory. The process involves the following steps:

1. **Load the Trained Generator Model**: Load the trained model that translates photos to Monet-style paintings.
2. **Process the Photos**: Load and preprocess the photos from the given path.
3. **Generate Monet-Style Images**: Use the generator to convert the photos.
4. **Save the Outputs**: Save the generated images to a specified directory.
5. **Create a .zip File**: Compress the directory into a .zip file for easy download.

Let's walk through these steps with detailed explanations and code.

### Step 1: Load the Trained Generator Model

Assume that you have trained and saved your generator model. Here’s how you can load it:

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os
import shutil

# Define the Generator model class
class Generator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(256) for _ in range(6)]
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=7, stride=1, padding=3),
            nn.Tanh()
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.res_blocks(x)
        x = self.decoder(x)
        return x

# Residual Block class
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual
        return out

In [1]:
# Load the trained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G_XtoY = Generator(in_channels=3, out_channels=3).to(device)
G_XtoY.load_state_dict(torch.load('/kaggle/input/monetgen/pytorch/monetgen_v1/1/G_XtoY.pth'))
G_XtoY.eval()

Generator(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (res_blocks): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

### Step 2: Process the Photos

Load and preprocess the photos to match the input format expected by the generator.

In [2]:
import os
from torchvision import transforms
from PIL import Image
import numpy as np

# Path to the directory containing photos
photos_path = "/kaggle/input/gan-getting-started/photo_jpg"

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load and preprocess photos
photo_filenames = [f for f in os.listdir(photos_path) if f.endswith('.jpg')]
photos = []
for filename in photo_filenames:
    image = Image.open(os.path.join(photos_path, filename)).convert('RGB')
    image = transform(image)
    photos.append(image)

photos = torch.stack(photos)


In [4]:
photos[:5].shape

torch.Size([5, 3, 256, 256])

### Step 3: Generate Monet-Style Images

Use the loaded generator model to convert the photos to Monet-style paintings.


#### Requirements from the competetion:

**Submission File** 
    You are going to generate 7,000-10,000 Monet-style images that are in jpg format. Their sizes should be 256x256x3 (RGB). Then you need to zip those images and your output from your Kernel should only have ONE output file named images.zip.

In [7]:
# Directory to save the output images
output_dir = '/images'
os.makedirs(output_dir, exist_ok=True)

# Generate Monet-style images
with torch.no_grad():
    for i, photo in enumerate(photos):
        photo = photo.unsqueeze(0).to(device)  # Add batch dimension
        monet_style_image = G_XtoY(photo)
        monet_style_image = monet_style_image.squeeze(0).cpu()  # Remove batch dimension

        # Convert tensor to PIL Image and denormalize
        monet_style_image = transforms.ToPILImage()(monet_style_image * 0.5 + 0.5)
        monet_style_image.save(os.path.join(output_dir, f'monet_style_{i+1}.jpg'))


### Step 4: Create a .zip File

Zip the output directory containing the Monet-style images.

In [8]:
import shutil

Generated Monet-style images and created a zip file successfully.


In [17]:
output_dir

'/images'

In [20]:
# Make Zipfile
shutil.make_archive("/kaggle/working/images", 'zip', output_dir)

'/kaggle/working/images.zip'

In [ ]:

print("Generated Monet-style images and created a zip file successfully.")

In [21]:
# Delete Origin file
shutil.rmtree(output_dir)

This code will process the photos, generate Monet-style images, save them in the specified directory, and create a .zip file of the output directory. You can now easily download the .zip file and share the results.